In [6]:
#import libraries
import os
import glob
import rasterio
import numpy as np
import matplotlib.pyplot as plt
import json

In [ ]:
# Define the folder containing the images
image_folder = 'DJI_202310171502_005_Create-Area-Route1'
# Initialize a dictionary to store the results for all images
results = {}

In [ ]:
# Function to interpret health based on mean values
def interpret_health(ndvi_value, gndvi_value, rendvi_value, reci_value):
    health_status = []
    
    # NDVI Interpretation
    if ndvi_value < 0:
        health_status.append("Non-vegetative surface (NDVI)")
    elif 0 <= ndvi_value < 0.3:
        health_status.append("Unhealthy or sparse vegetation (NDVI)")
    elif 0.3 <= ndvi_value < 0.6:
        health_status.append("Moderate to healthy vegetation (NDVI)")
    elif ndvi_value >= 0.6:
        health_status.append("Very healthy, dense vegetation (NDVI)")
    
    # GNDVI Interpretation (Chlorophyll/Nitrogen)
    if gndvi_value < 0.1:
        health_status.append("Very low chlorophyll content (GNDVI)")
    elif 0.1 <= gndvi_value < 0.5:
        health_status.append("Moderate chlorophyll content (GNDVI)")
    elif gndvi_value >= 0.5:
        health_status.append("High chlorophyll content (GNDVI)")

    # RENDVI Interpretation (Early Stress Detection)
    if rendvi_value < 0.2:
        health_status.append("Early signs of stress (RENDVI)")
    elif 0.2 <= rendvi_value < 0.5:
        health_status.append("Moderate stress (RENDVI)")
    elif rendvi_value >= 0.5:
        health_status.append("Healthy vegetation, no stress (RENDVI)")

    # RECI Interpretation (Chlorophyll Concentration)
    if reci_value < 2:
        health_status.append("Low chlorophyll concentration (RECI)")
    elif 2 <= reci_value < 5:
        health_status.append("Moderate chlorophyll concentration (RECI)")
    elif reci_value >= 5:
        health_status.append("High chlorophyll concentration (RECI)")

    return "; ".join(health_status)

In [ ]:
# Create a list of all the JPG images in the folder
jpg_images = glob.glob(os.path.join(image_folder, '*_D.jpg'))

# Process each JPG image and its corresponding multispectral images
for jpg_image in jpg_images:
    # Extract the base name without extensions (e.g., 'imagename')
    filename = os.path.basename(jpg_image)
    base_name = filename.rsplit('_', 1)[0]

    # Paths to the corresponding multispectral images
    nir_path = os.path.join(image_folder, f'{base_name}_MS_NIR.TIF')
    red_path = os.path.join(image_folder, f'{base_name}_MS_R.TIF')
    green_path = os.path.join(image_folder, f'{base_name}_MS_G.TIF')
    re_path = os.path.join(image_folder, f'{base_name}_MS_RE.TIF')
    print(nir_path)

    # Check if all required multispectral images exist
    if not all([os.path.exists(nir_path), os.path.exists(red_path), 
                os.path.exists(green_path), os.path.exists(re_path)]):
        print(f"Missing multispectral images for {base_name}. Skipping...")
        continue

    # Open the NIR, Red, Green, and Red-Edge band images using rasterio
    with rasterio.open(nir_path) as nir_src:
        nir_band = nir_src.read(1)

    with rasterio.open(red_path) as red_src:
        red_band = red_src.read(1)

    with rasterio.open(green_path) as green_src:
        green_band = green_src.read(1)

    with rasterio.open(re_path) as re_src:
        re_band = re_src.read(1)

    # Convert to float to avoid issues with integer division
    nir_band = nir_band.astype(float)
    red_band = red_band.astype(float)
    green_band = green_band.astype(float)
    re_band = re_band.astype(float)

    # Avoid division by zero by using np.errstate
    with np.errstate(divide='ignore', invalid='ignore'):
        # Calculate NDVI
        ndvi = (nir_band - red_band) / (nir_band + red_band)
        ndvi[np.isnan(ndvi)] = 0

        # Calculate GNDVI
        gndvi = (nir_band - green_band) / (nir_band + green_band)
        gndvi[np.isnan(gndvi)] = 0

        # Calculate RENDVI
        rendvi = (nir_band - re_band) / (nir_band + re_band)
        rendvi[np.isnan(rendvi)] = 0

        # Calculate RECI
        reci = (nir_band / re_band) - 1
        reci[np.isnan(reci)] = 0

    # Calculate statistics for each index
    ndvi_mean = np.mean(ndvi)
    gndvi_mean = np.mean(gndvi)
    rendvi_mean = np.mean(rendvi)
    reci_mean = np.mean(reci)

    # Interpret the overall plant health
    crop_health = interpret_health(ndvi_mean, gndvi_mean, rendvi_mean, reci_mean)

    # Store the results in the dictionary
    results[base_name] = {
        "mean": {
            "NDVI": ndvi_mean,
            "GNDVI": gndvi_mean,
            "RENDVI": rendvi_mean,
            "RECI": reci_mean,
        },
        "std": {
            "NDVI": np.std(ndvi),
            "GNDVI": np.std(gndvi),
            "RENDVI": np.std(rendvi),
            "RECI": np.std(reci),
        },
        "crop_health_description": crop_health
    }
    
    
    # Create a directory for the image outputs
    output_dir = os.path.join(image_folder, base_name)
    os.makedirs(output_dir, exist_ok=True)

    # Save the output images as new TIF files
    output_paths = {
        os.path.join(output_dir, "NDVI.TIF"): ndvi,
        os.path.join(output_dir, "GNDVI.TIF"): gndvi,
        os.path.join(output_dir, "RENDVI.TIF"): rendvi,
        os.path.join(output_dir, "RECI.TIF"): reci,
    }

    for path, data in output_paths.items():
        with rasterio.open(path, 'w',
                           driver='GTiff',
                           height=data.shape[0], width=data.shape[1],
                           count=1, dtype=data.dtype) as dst:
            dst.write(data, 1)

    print(f"Processed and saved indices for {base_name} in {output_dir}.")

In [7]:
# Save the results to a JSON file
json_output_path = os.path.join(image_folder, "image_health_analysis.json")
with open(json_output_path, 'w') as json_file:
    json.dump(results, json_file, indent=4)

print(f"Processing complete. Results saved to {json_output_path}")

Processing complete. Results saved to DJI_202310171502_005_Create-Area-Route1\image_health_analysis.json
